In [ ]:
using Pkg
Pkg.activate("."); Pkg.instantiate()

In [ ]:
#import Pkg
Pkg.add("AxisArrays")
Pkg.add("JuMP")
Pkg.add("XLSX")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("AxisArrays")
Pkg.add("Statistics")
Pkg.add("PGFPlotsX")
Pkg.add("AxisArrays")
Pkg.add("CPLEX")
Pkg.add("HiGHS")

In [ ]:
using JuMP, XLSX, DataFrames, Plots, AxisArrays, Statistics, PGFPlotsX, CPLEX, HiGHS

In [ ]:
pgfplotsx()

# Read In And Prepare Data 

In [ ]:
mydata = Dict{String, DataFrame}()

In [ ]:
xl = XLSX.readxlsx("data/curtailmentinfo.xlsx")
mydata["curtailmentinfo"] = DataFrame(XLSX.gettable(xl["curtailmentinfo"]; infer_eltypes=true))
print("curtailment data read in successfully")

In [ ]:
mydata["curtailmentinfo"]

In [ ]:
xl = XLSX.readxlsx("data/generatorinfo.xlsx")
generatorinfo = XLSX.sheetnames(xl)
mydata["generatorinfo"] = DataFrame(XLSX.gettable(xl["generatorinfo"]; infer_eltypes=true))
print("generator data read in successfully")

In [ ]:
mydata["generatorinfo"]

In [ ]:
xl = XLSX.readxlsx("data/initdemand.xlsx")
initdemand = XLSX.sheetnames(xl)
mydata["initdemand"] = DataFrame(XLSX.gettable(xl["initdemand"]; infer_eltypes=true))
print("demand data read in successfully")

In [ ]:
mydata["initdemand"]

In [ ]:
xl = XLSX.readxlsx("data/lineconstraints.xlsx")
linecontraints = XLSX.sheetnames(xl)
mydata["lineconstraints"] = DataFrame(XLSX.gettable(xl["lineconstraints"]; infer_eltypes=true))
print("line constraint data read in successfully")

In [ ]:
mydata["lineconstraints"]

In [ ]:
xl = XLSX.readxlsx("data/nodeinfo.xlsx")
nodeinfo = XLSX.sheetnames(xl)
mydata["nodeinfo"] = DataFrame(XLSX.gettable(xl["nodeinfo"]; infer_eltypes=true))
print("node data read in successfully")

In [ ]:
mydata["nodeinfo"]

In [ ]:
xl = XLSX.readxlsx("data/shiftinginfo.xlsx")
shiftinginfo = XLSX.sheetnames(xl)
mydata["shiftinginfo"] = DataFrame(XLSX.gettable(xl["shiftinginfo"]; infer_eltypes=true))
print("shifting information read in successfully")

In [ ]:
mydata["shiftinginfo"]

In [ ]:
xl = XLSX.readxlsx("data/windfarminfo.xlsx")
windfarminfo = XLSX.sheetnames(xl)
mydata["windfarminfo"] = DataFrame(XLSX.gettable(xl["windfarminfo"]; infer_eltypes=true))
print("windfarm data read in successfully")

In [ ]:
mydata["windfarminfo"]

In [ ]:
xl = XLSX.readxlsx("data/windscenarios.xlsx")
windscenarios = XLSX.sheetnames(xl)
mydata["windscenarios"] = DataFrame(XLSX.gettable(xl["windscenarios"]; infer_eltypes=true))
print("wind scenario data read in successfully")


In [ ]:
mydata["windscenarios"]

### Set generation

In [ ]:
G = mydata["generatorinfo"][:, :ID]
L = mydata["lineconstraints"][:, :ID]
T = mydata["initdemand"][:, :t]          #times
N = mydata["nodeinfo"][:, :ID]
S = unique(mydata["windscenarios"][:, :scenario])
K = unique(mydata["curtailmentinfo"][:, :k])

NG = length(G)
NL = length(L)
NT = length(T)
NN = length(N)
NS = length(S)
NK = length(K)

# Prepare data

Prepare the generator information

In [ ]:
#generatorinfo
generators = Dict()
for c in names(mydata["generatorinfo"])
    arr = AxisArray(mydata["generatorinfo"][:, c], g=G)
    generators[c] = arr
end

#other version
generators = Dict{String, AxisArray}()
for c in names(mydata["generatorinfo"])
    arr = AxisArray(mydata["generatorinfo"][:, c], g=G)
    generators[c] = arr
end

In [ ]:
#new code for generator dict
generators = {c => AxisArray(mydata["generatorinfo"][:, c], g=G) for c in names(mydata["generatorinfo"])}


Prepare all of the line constraints

In [ ]:
#lineconstraints
lines = Dict(c => AxisArray(mydata["lineconstraints"][:, c], l=L) for c in names(mydata["lineconstraints"]));
B = AxisArray(zeros(NL, NN), l=L, n=N)

# new version
# # create AxisArray of line constraints
# lines = Dict()
# for c in names(mydata["lineconstraints"])
#     lines[c] = AxisArray(mydata["lineconstraints"][:, c], l=L)

# # initialize B matrix
# B = AxisArray(zeros(NL, NN), l=L, n=N)

for l in L
    from_node = lines["from"][l]
    to_node = lines["to "][l]
    X = lines["X"][l]
    for (idn, node) in enumerate(N)
        if node == from_node
            B[l, idn] = -1/X
        elseif node == to_node
            B[l, idn] = 1/X
        end
    end
end 


Prepare the data of the initial demand (demand before DR)

In [ ]:
#initdemand
load = AxisArray(Matrix(mydata["initdemand"][:, Not(:t)]), t=T, n=N)
cost_ls = 100.0

Prepare the load shifiting information

In [ ]:
#shiftinginfo
cols = names(mydata["shiftinginfo"][:, Not(["t", "n", "k"])])
DRSd = Dict(c => AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K) for c in cols)

# # Create a dictionary DRSd with keys from cols
# DRSd = Dict()
# for c in cols
#     # Create an AxisArray filled with zeros with dimensions NT x NN x NK
#     arr = AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K)
#     # Add the AxisArray to the dictionary using the key c
#     DRSd[c] = arr
# end



for r in eachrow(mydata["shiftinginfo"])
    for c in cols
        DRSd[c][r[:t], r[:n], r[:k]] = r[c]
    end
end
max_rec = 2.0

Prepare the load curtailment information

In [ ]:
#curtailmentinfo
cols = names(mydata["curtailmentinfo"][:, Not(["t", "n", "k"])])

DRCd = Dict(c => AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K) for c in cols)

for r in eachrow(mydata["curtailmentinfo"])
    for c in cols
        DRCd[c][r[:t], r[:n], r[:k]] = r[c]
    end
end


In [ ]:
DRCd

Prepare the information of the location of the nodes and generators

In [ ]:
#location
ng = AxisArray(
    [[g for g in G if generators["location"][g] == idn] for (idn, n) in enumerate(N)],
    n = N
)

#node
nl = AxisArray(zeros(Int, NN, NL), n=N, l=L)
for l in L
    nl[N[lines["from"][l]], l] = 1
    nl[N[lines["to "][l]], l] = -1
end

In [ ]:
wind_cap = AxisArray(zeros(NN), n=N)
wind_cap["n3"] = 500.0
wind_cap["n5"] = 500.0
wind_cap["n16"] = 300.0
wind_cap["n21"] = 300.0

pi = AxisArray(ones(NS)/NS, s=S)

wind = AxisArray(zeros(NS, NN, NT), s=S, n=N, t=T)
for r in eachrow(mydata["windscenarios"])
    for t in T
        wind[r[:scenario], r[:node], t] = r[t]*wind_cap[r[:node]]
    end
end

ewind = mean(wind, dims=1)[1, :, :]
stdwind = std(wind, dims=1)[1, :, :]
cost_ws = 20.0

In [ ]:
pi

# Model

The model is intended to represent an electric power system with wind power and demand response (DR) programs.



In [ ]:
bdr = Model(CPLEX.Optimizer)

@variables(bdr, begin
    pgen[G, T] >= 0 # scheduled gen. output
    u[G, T], Bin # state of generator
    θ0[N, T] # phase angles in scheduling phase
    θ1[N, S, T] # phase angles in balancing phase
    Wsc[N, T] >= 0# scheduled wind power
    r_up[G, T] >= 0 # up regulation
    r_down[G, T] >= 0 # down regulation
    r_ups[S, G, T] >= 0 # actual up regulation
    r_downs[S, G, T] >= 0 # actual down regulation 
    Lshed[S, N, T] >= 0 # load shedding
    Wspill[S, N, T] >= 0 # wind spillage
    suc[1:NT, G] >= 0 # generator start-up/shut-down cost

    # DR contracts
    # DR agg <> ISO contract parameters
    DRCK[T, N, K] >= 0 # DR curtailment blocks
    DRSK[T, N, K] >= 0 # DR shifting blocks
    DRC_SU[T, N, K] >= 0 # DRC startup cost
    DRSRK[T, N, K] >= 0 # DR recovery blocks
    uDRC[T, N, K], Bin # DR curtailment active
    uDRS[T, N, K], Bin # DR shifting activate
    yDR[T, N, K], Bin # DRC activated
    zDR[T, N, K], Bin # DRC de-activated
    uDRSR[T, N, K], Bin # DRS recovery active
    # same for DR agg <> cust. contracts
    DBCK[T, N, K] >= 0
    DBSK[T, N, K] >= 0
    DBC_SU[T, N, K] >= 0 
    DBSRK[T, N, K] >= 0
    uDBC[T, N, K], Bin
    uDBS[T, N, K], Bin
    yDB[T, N, K], Bin
    zDB[T, N, K], Bin
    uDBSR[T, N, K], Bin
    # Dual multipliers/linearization variables for KKT formulation
    A1[T,N,K] >= 0 
    B1[T,N,K] >= 0
    D1[T,N,K] >= 0
    E1[T,N,K] >= 0
    F1[T,N,K] >= 0
    H1[T,N,K] >= 0
    I1[T,N,K] >= 0
    K1[T,N,K] >= 0
    L1[T,N,K] >= 0
    M1[T,N,K] >= 0
end)

# fix slack bus
fix.(θ0["n1", :], 0);
fix.(θ1["n1", :, :], 0);

@expressions(bdr, begin
    # DR costs/profits
    DRC[t in T, n in N], sum(DRCd["cost"][t,n,k]*DRCK[t, n, k]+DRC_SU[t,n,k] for k in K)
    DRS[t in T, n in N], sum(DRSd["cost"][t,n,k]*DRSK[t, n, k] for k in K)
    DBC[t in T, n in N], sum(DRCd["costb"][t,n,k]*DBCK[t, n, k] for k in K)
    DBS[t in T, n in N], sum(DRSd["costb"][t,n,k]*DBSK[t, n, k] for k in K)
    pflow0[l in L, t in T], sum(B[l,n]*θ0[n,t] for n in N)
    pflow1[l in L, s in S, t in T], sum(B[l,n]*θ1[n,s,t] for n in N)
    totalcost, 
        sum(
            sum(
                pgen[g,t] * generators["cost"][g] +
                r_up[g,t] * generators["upresercost"][g] +
                r_down[g,t] * generators["dnresercost"][g] +
                suc[findfirst(isequal(t), T),g]
            for g in G) +
            sum(DRC[t, n] + DRS[t, n] for n in N) +
            sum(
                pi[s] * (
                    sum(generators["cost"][g] * (r_ups[s,g,t]-r_downs[s,g,t]) for g in G) +
                    sum(cost_ws * Wspill[s,n,t] + cost_ls * Lshed[s,n,t] for n in N)
                    )
            for s in S)
        for t in T)
end)

# more understandable version of this loop 
# total_cost = 0.0
# for t in T
#     generator_cost = 0.0
#     reserve_cost = 0.0
#     start_cost = 0.0
    
#     # Generator cost
#     for g in G
#         generator_cost += pgen[g,t] * generators["cost"][g] +
#                             r_up[g,t] * generators["upresercost"][g] +
#                             r_down[g,t] * generators["dnresercost"][g] +
#                             suc[findfirst(isequal(t), T),g]
#     end
    
#     # Reserve cost
#     for n in N
#         reserve_cost += DRC[t, n] + DRS[t, n]
#     end
    
#     # Startup cost
#     for s in S
#         startup_cost = 0.0
#         for g in G
#             startup_cost += generators["cost"][g] * (r_ups[s,g,t]-r_downs[s,g,t])
#         end
#         start_cost += pi[s] * (startup_cost + sum(cost_ws * Wspill[s,n,t] + cost_ls * Lshed[s,n,t] for n in N))
#     end
    
#     # Total cost
#     total_cost += generator_cost + reserve_cost + start_cost
# end




@constraints(bdr, begin
    # generator limits
    pup[g in G, t in T], pgen[g, t] + r_up[g, t] <= generators["cap"][g] * u[g,t]
    res_up[g in G, t in T], r_up[g, t] <= generators["maxupresercap"][g]
    res_dn[g in G, t in T], r_down[g, t] <= generators["maxdnresercap"][g]
    # reserve limits  
    bal0[n in N, t in T], 
        sum(pgen[g,t] for g in ng[n]) + 
        Wsc[n,t] + 
        sum(DRCK[t,n,k] + DRSK[t,n,k] - DRSRK[t,n,k] for k in K) == 
        load[t,n] + sum(pflow0[l, t]*nl[n,l] for l in L)
    pfmax0[l in L, t in T], pflow0[l, t] <= lines["flowcap"][l]
    pfmin0[l in L, t in T], pflow0[l, t] >= -lines["flowcap"][l]
    # DR curtailment constraints (ISO <> DR agg.)
    drcmax[t in T, n in N, k in K], DRCK[t,n,k] <= uDRC[t,n,k] * DRCd["max"][t,n,k]
    drcmin[t in T, n in N, k in K], DRCK[t,n,k] >= uDRC[t,n,k] * DRCd["min"][t,n,k]
    drcst1[τ in 2:NT, n in N, k in K], yDR[T[τ],n,k] - zDR[T[τ],n,k] == uDRC[T[τ],n,k] - uDRC[T[τ-1],n,k]
    drcst2[t in T, n in N, k in K], yDR[t,n,k] + zDR[t,n,k] <= 1
    drcst3[t in T, n in N, k in K], DRC_SU[t,n,k] >= DRCd["SUC"][t,n,k] * yDR[t,n,k]
    # DR shifting constraints (ISO <> DR agg.)
    drsmax[t in T, n in N, k in K], DRSK[t,n,k] <= uDRS[t,n,k] * DRSd["max"][t,n,k]
    drsmin[t in T, n in N, k in K], DRSK[t,n,k] >= uDRS[t,n,k] * DRSd["min"][t,n,k]
    drsmxr[t in T, n in N, k in K], DRSRK[t,n,k] <= max_rec*uDRSR[t,n,k]
    drsorr[t in T, n in N, k in K], uDRS[t,n,k] + uDRSR[t,n,k] <= 1
    drbbal[n in N, k in K], sum(DRSRK[t,n,k] for t in T) == sum(DRSK[t,n,k] for t in T)
    # DR curt. cons (DR agg. <> cust.)
    dbcmax[t in T, n in N, k in K], DBCK[t,n,k] <= uDRC[t,n,k] * DRCd["max"][t,n,k]
    dbcmin[t in T, n in N, k in K], DBCK[t,n,k] >= uDRC[t,n,k] * DRCd["min"][t,n,k]
    dbcst1[τ in 2:NT, n in N, k in K], yDB[T[τ],n,k] - zDB[T[τ],n,k] == uDBC[T[τ],n,k] - uDBC[T[τ-1],n,k]
    dbcst2[t in T, n in N, k in K], yDB[t,n,k] + zDB[t,n,k] <= 1
    dbcst3[t in T, n in N, k in K], DBC_SU[t,n,k] >= DRCd["SUCb"][t,n,k] * yDB[t,n,k]
    # DR shift. cons (DR agg. <> cust.)
    dbsmax[t in T, n in N, k in K], DBSK[t,n,k] <= uDBS[t,n,k] * DRSd["max"][t,n,k]
    dbsmin[t in T, n in N, k in K], DBSK[t,n,k] >= uDBS[t,n,k] * DRSd["min"][t,n,k]
    dbsmxr[t in T, n in N, k in K], DBSRK[t,n,k] <= max_rec*uDBSR[t,n,k]
    dbsorr[t in T, n in N, k in K], uDBS[t,n,k] + uDBSR[t,n,k] <= 1
    dbbbal[n in N, k in K], sum(DBSRK[t,n,k] for t in T) == sum(DBSK[t,n,k] for t in T)
    # Dual multipliers/linearization constraints
    conAB[t in T, n in N, k in K], A1[t,n,k]-B1[t,n,k] >= DRCd["cost"][t,n,k]
    conDE[t in T, n in N, k in K], D1[t,n,k]-E1[t,n,k] >= DRSd["cost"][t,n,k]
    conHI[t in T, n in N, k in K], H1[t,n,k]-I1[t,n,k] >= -DRCd["costb"][t,n,k]
    conKL[t in T, n in N, k in K], K1[t,n,k]-L1[t,n,k] >= -DRSd["costb"][t,n,k]
    strong_duality, 
        sum(
            A1[t,n,k]*DRCd["max"][t,n,k] - B1[t,n,k]*DRCd["min"][t,n,k] +
            D1[t,n,k]*DRSd["max"][t,n,k] - E1[t,n,k]*DRSd["min"][t,n,k] +
            F1[t,n,k]*max_rec +
            H1[t,n,k]*DRCd["max"][t,n,k] - I1[t,n,k]*DRCd["min"][t,n,k] +
            K1[t,n,k]*DRSd["max"][t,n,k] - L1[t,n,k]*DRSd["min"][t,n,k] +
            M1[t,n,k]*max_rec
        for t in T for n in N for k in K) ==
        sum(
            DRCd["cost"][t,n,k] * DRCK[t,n,k] +
            DRSd["cost"][t,n,k] * DRSK[t,n,k] - 
            DRCd["costb"][t,n,k] * DBCK[t,n,k] - 
            DRSd["costb"][t,n,k] * DBSK[t,n,k]    
        for t in T for n in N for k in K)
    # second stage (after wind realizes)
    resup[s in S, g in G, t in T], r_ups[s,g,t] <= r_up[g,t]
    resdn[s in S, g in G, t in T], r_downs[s,g,t] <= r_down[g,t]
    bals[s in S, n in N, t in T], 
        sum(r_ups[s,g,t]-r_downs[s,g,t] for g in ng[n]) + 
        wind[s,n,t] - Wsc[n,t] - Wspill[s,n,t] + Lshed[s,n,t] == 
        -sum((pflow1[l,s,t] - pflow0[l,t])*nl[n,l] for l in L)
    windspill[s in S, n in N, t in T], Wspill[s,n,t] <= wind[s,n,t]
    pfmax1[l in L, s in S, t in T], pflow1[l,s,t] <= lines["flowcap"][l]
    pfmin1[l in L, s in S, t in T], pflow1[l,s,t] >= -lines["flowcap"][l]
end)
    
@objective(bdr, Min, totalcost)

bdr

# Model

In [ ]:
optimize!(bdr)

In [ ]:
termination_status(bdr)

In [ ]:
objective_value(bdr)

In [ ]:
value.(pgen)

In [ ]:
value.(u)

In [ ]:
B = sum(load, dims=2)[:, 1]
B

In [ ]:
A = [sum(load[t,:]) - sum(value.(DRCK[t,:,:]) .+ value.(DRSK[t,:,:]) .- value.(DRSRK[t,:,:])) for t in T]
A

In [ ]:
A 
B 
using XLSX

# Create a new workbook and add a worksheet
wb = XLSX.Workbook()
wb = XLSX.writeworkbook("BilevelResponseData.xlsx")
ws = XLSX.addworksheet(wb, "Sheet1")

# Write the data to the worksheet
XLSX.writetable(ws, [A B])

# Save the workbook
XLSX.closexlsx(wb)


In [ ]:
plot!(
    sum(load, dims=2)[:, 1], 
    label="Load before DR",
    xlims=(1,24),
    xticks=6:6:24,
    )

plot!(
    [sum(load[t,:]) - sum(value.(DRCK[t,:,:]) .+ value.(DRSK[t,:,:]) .- value.(DRSRK[t,:,:])) for t in T], 
    label="Load after DR"
    )

print("test")

In [ ]:
plot(
    ewind["n6", :], 
    ribbon = stdwind["n6", :],
    label="μw +- σw",
    xlims=(1,24),
    xticks=6:6:24,
    )